In [1]:
## imports
import numpy as np
from sklearn.model_selection import (
    StratifiedKFold,  # use X = np.zeros(n_samples) in .split
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

from data_img import Img_VAE_Dataset

# if you have indeces and want knns from large set, can make an idx array and NearestVectorCaller._call_vec_set; can just train/test split np.arange(len(ds)) to get back indeces for large sets


from btv import *

In [2]:
## Data setup

ds = Img_VAE_Dataset("data/hardhat/test", max_dim=128)
# can pass metric == "cosine", get from NearestVectorCaller.metric on large sets
knn = KNeighborsClassifier(weights="distance", n_neighbors=10) #, metric = "cosine")
# X, y = ds.whole_img_vec_set[:] # only need y for class balancing... ds.whole_img_vec_set.y

idx_train, idx_test = train_test_split(
    np.arange(len(ds)), test_size=0.2
)
(X_train, y_train), (X_test, y_test) = ds.whole_img_vec_set[idx_train], ds.whole_img_vec_set[idx_test]

Detemining all classes...: 100%|██████████████████████████████| 1766/1766 [00:00<00:00, 2084.45it/s]


AttributeError: 'Img_VAE_Dataset' object has no attribute 'whole_img_vec_set'

In [7]:
knn.fit(X_train, y_train)
# ds.whole_img_vec_set.X[idx_test]
knn.score(X_test,y_test)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [10]:
accuracy_score(
    ds.whole_img_vec_set.y[idx_test], knn.predict(ds.whole_img_vec_set.X[idx_test])
)

0.8107344632768362

In [9]:
# only include p if y==1 or, reeally, want p(right answer) so take elemnt of predicted prob indicated by binary y.
def prediction_entropy(y_true, y_predicted): # multilable
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    A = np.zeros_like(y_true)
    with np.errstate(divide="ignore"):
        for i, (pred, true) in enumerate(zip(y_predicted, y_true)):
            for j, (p, t) in enumerate(zip(pred, true)):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                A[i, j] = I
    return np.mean(A, axis=0)


# probably actually want to do this at the prediction level since that is the big/slow bit.
def prediction_entropy_lg(y_true, y_predicted): # sinlge label
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    with np.errstate(divide="ignore"):
        for pred, true in zip(y_predicted, y_true):
            for p, t in zip(pred, true):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                yield I

In [40]:
A = knn.predict(ds.whole_img_vec_set.X[idx_test])
P = knn.predict_proba(
    ds.whole_img_vec_set.X[idx_test]
)  # predicts the probability of each class. For whole image hardhats it is a list of 3 (p(no), P(yes)) arrays

In [41]:
prediction_entropy(y_predicted, y_true)

AttributeError: 'list' object has no attribute 'shape'

In [17]:
kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
# can just add all the labels to make it multiclass for splitting
entropies = []
running_train = []
for k, (train_idx, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
    # knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    # y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[test_idx])
    # y_true = ds.whole_img_vec_set.y[test_idx]
    # H = prediction_entropy(y_true, y_predicted)
    # entropies.append(H)
    # print(f"Fold : {k+1}, " f"Fold entropy : {H}")

In [20]:
np.mean(entropies, axis=0), np.std(entropies, axis=0), np.std(
    entropies, axis=0
) / np.mean(entropies, axis=0)

(array([75.04532578, 57.61076487, 16.80113314]),
 array([38.20580196, 28.99206553,  8.40056667]),
 array([0.50910302, 0.50324042, 0.50000001]))

In [188]:
knn.predict(ds.whole_img_vec_set.X[test_idx])

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [100]:
np.sum(ds.whole_img_vec_set.y, axis=0)

array([ 339, 1604,   45])

In [43]:
## with straigh vectorization of whole images, the knn accuracy goes down as more data is added. Therefore that vecotrization is shit. Maybe we try tabular data?

kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
train_idx = np.array([], int)
running_train = []
for k, (_, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
running_test = running_train.pop()
for k, old_test_idx in enumerate(running_train):
    train_idx = np.append(train_idx,old_test_idx)
    knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[running_test])
    y_true = ds.whole_img_vec_set.y[running_test]
    H = prediction_entropy(y_true, y_predicted)
    entropies.append(H)
    print(f"Fold : {k+1}, " f"Test set entropy : {np.mean(H)}", f"Train samples : {len(train_idx)}", f"Accuracy : {accuracy_score(y_true,knn.predict(ds.whole_img_vec_set.X[test_idx]))}")

Fold : 1, Test set entropy : 45.036827195467424 Train samples : 354 Accuracy : 0.7705382436260623
Fold : 2, Test set entropy : 45.040604343720496 Train samples : 707 Accuracy : 0.7648725212464589
Fold : 3, Test set entropy : 47.038715769593956 Train samples : 1060 Accuracy : 0.7620396600566572
Fold : 4, Test set entropy : 52.03305004721435 Train samples : 1413 Accuracy : 0.7592067988668555


In [42]:
hs, ss = add_stratified_folds_test(ds.whole_img_vec_set.X,ds.whole_img_vec_set.y,knn)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [4]:
imc = ds.imc[:100]
color = len(imc[0].shape) > 2
channels = imc[0].shape[2] if color else 1
    
px_mean, px_std = np.zeros(channels), np.zeros(channels)
running_sum,running_sumsq  = 0,0 
for n, im in tqdm(
    enumerate(imc), desc="Scanning to find mean and STD of pixel values...", unit="Image file"
):

    last_px_mean = px_mean.copy()
    last_px_std = px_std.copy()
    px_mean += (im.mean(0).mean(0) - px_mean)/(n + 1)
    px_std = (im.std(0).std(0) - last_px_std)/(n + 1)

    print(im.std(0).std(0), np.power(px_std,0.25))

Scanning to find mean and STD of pixel values...: 26Image file [00:00, 252.12Image file/s]

[1.2318113 1.4534428 1.6422439] [1.05350365 1.09799279 1.13203385]
[1.6700602 1.7795748 1.770059 ] [0.68418367 0.63546383 0.50279166]
[1.4173375 1.244182  1.5806733] [0.79497451 0.7747967  0.84323634]
[1.5912611 1.7805898 1.9391171] [0.73882404 0.77192257 0.77372492]
[1.7403489 1.8966837 2.3457093] [0.73287141 0.74516506 0.79400794]
[1.740151  1.7675205 1.8884574] [0.70134139 0.70224798 0.70604278]
[2.1762805 2.540266  2.7033827] [0.72503405 0.75686584 0.76954361]
[1.9214222 1.9743167 2.2817085] [0.67340267 0.67351271 0.70092843]
[2.0022497 2.217053  2.7195246] [0.66842555 0.6875553  0.72438791]
[1.739645  1.8590896 2.1123595] [0.62644317 0.63594593 0.65467818]
[1.5354294 1.7255011 1.8372145] [0.59529719 0.61385796 0.62266401]
[1.4350318 1.8585143 2.4046593] [0.574747   0.61498836 0.65835409]
[1.6912606 1.8210794 2.0635042] [0.59064344 0.59939689 0.6163135 ]
[0.93127114 0.98015565 1.2459649 ] [0.4903784  0.49654667 0.52964188]
[1.938669  1.9799638 2.1793985] [0.59506623 0.59807507 0.61

Scanning to find mean and STD of pixel values...: 100Image file [00:00, 250.21Image file/s]

[1.8751795 1.7735482 1.7269161] [0.43367506 0.4273725  0.42403932]
[1.4566951 1.6566691 1.9193112] [0.40467302 0.41834188 0.43438302]
[1.8546876 1.9543949 2.3294976] [0.42893145 0.43444985 0.45398856]
[1.5723968 1.6128262 1.7960677] [0.40896585 0.41151065 0.42256277]
[1.5420915 1.3962276 1.9071897] [0.40550192 0.39531099 0.42778056]
[2.2680368 2.442622  2.7910063] [0.44528873 0.45384171 0.46898711]
[1.8816338 2.0341432 2.2751508] [0.42216687 0.43047704 0.44265156]
[1.924601  1.9740739 2.1826773] [0.42321913 0.42581655 0.436624  ]
[1.5619034 1.4240656 1.8282518] [0.39959714 0.39021926 0.41571062]
[1.3886795 1.5259602 1.805834 ] [0.38663934 0.39617853 0.41307259]
[1.8811905 2.118001  2.361266 ] [0.41611443 0.42866037 0.44039388]
[1.3134642 1.4211087 1.6223089] [0.37780037 0.38522189 0.39824578]
[2.1004784 2.1697516 2.3953824] [0.42459645 0.42800608 0.43868478]
[1.5756496 1.5196352 1.7781814] [0.39253063 0.38884989 0.40455772]
[1.7674989 1.8518579 2.1095457] [0.40316756 0.40800657 0.42147

NameError: name 'y_predicted' is not defined